#### Détourage des cellules avec recadrage et non redimensionnement => 256x256

> Choix d'enregistrer les images des cellules détourées pour modèle de prédiction de type 
>
> Test réalisé de détourage via prédiction par le modèle de segmentation au moment de l'entrainement d'un modèle de prédiction : temps d'entrainement trop long

In [1]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import cv2

In [2]:
data = pd.read_csv('../base_10_class.csv',index_col=0)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31735 entries, 0 to 31734
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_path  31735 non-null  object
 1   filename   31735 non-null  object
 2   class      31735 non-null  object
dtypes: object(3)
memory usage: 991.7+ KB


In [3]:
from keras.models import load_model
model_mask = load_model('../4_ajout_donnees_et_segmentation/segmentation_fcn')

In [4]:
# Process pour detourage
# Fonction preprocess pour les images à detourer (modèle de masque)
def load_and_preprocess_mask(img_filepath):
    img = tf.io.read_file(img_filepath)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [128, 128]) 
    img = tf.cast(img, tf.float32) / 255.0
    return img

# Création du masque à partir de la prédiction, mise au format nécessaire
def create_mask(pred_mask,resize):
    mask = tf.argmax(pred_mask, axis=-1) 
    mask = mask[..., tf.newaxis]
    mask = tf.image.resize(mask,resize)
    return tf.squeeze(mask[0],-1)

# Detourage de la cellule en utilisant la prédiction du modèle de masque
def cut_contour(img_filepath,model,resize):
    img_to_mask = load_and_preprocess_mask(img_filepath)
    pred_mask = model(img_to_mask[tf.newaxis, ...])
    img = tf.io.read_file(img_filepath)
    img = tf.io.decode_jpeg(img, channels=3)
    shape = tf.shape(img)
    mask = create_mask(pred_mask,(shape[0],shape[1]) )
    img = tf.cast(img, tf.float32)
    # Supprime autour de la cellule => noir
    mask_bg = (mask == 0)[..., tf.newaxis]
    mask_bg  = tf.cast(mask_bg, tf.float32)
    mask_bg  = tf.concat([mask_bg, mask_bg, mask_bg], 2)
    img_res = img *  mask_bg
    # Recadrage de l'image centrée
    img_res = tf.image.resize_with_crop_or_pad(img_res, resize[0], resize[1])
    return ( img_res)

In [5]:
def load_and_preprocess(filename, model, resize=(256,256)):
    # Detourage
    img = cut_contour(filename,model,resize)
    return img

In [6]:
AUTO = tf.data.experimental.AUTOTUNE

In [7]:
dataset = tf.data.Dataset.from_tensor_slices((data['file_path'],data['filename']))
dataset = (dataset.map(lambda x, y: [load_and_preprocess(x, model_mask),y], num_parallel_calls=AUTO))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [10]:
#!mkdir ..\images_detour # Windows
!mkdir ../images_detour_256

In [11]:
dest_img = '../images_detour_256/'

In [12]:
# Fonction pour boucher les trous si existe dans l'image
# Dans background, pixel autre que noir
# Dans la cellule, pixel noir à remplacer
def morpho(img):
    gray = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_RGB2GRAY)
    # Tous les pixels non noirs => blanc
    seuil,img_threshold = cv2.threshold(gray,  1,255, cv2.THRESH_BINARY )
    # Bouche les trous du background 
    seuil,img_threshold = cv2.threshold(gray,  1,255, cv2.THRESH_BINARY )
    kernel = np.ones((5,5),np.uint8)
    opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    gray_opening = cv2.cvtColor(opening.astype(np.uint8), cv2.COLOR_RGB2GRAY)
    seuil,opening_threshold = cv2.threshold(gray_opening,  1,1, cv2.THRESH_BINARY )
    bg  = cv2.merge((opening_threshold,opening_threshold,opening_threshold))
    img =bg*img
    # Remplit les trous de la cellule (point noir dans la cellule)
    closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    img_black = (gray==0).astype(int)
    img_black = cv2.merge((img_black,img_black,img_black))
    img_out = ( img_black*closing) + img
    return img_out

In [13]:
# Enregistrement des images détourées sans sous dossier
for image, filename in dataset.take(-1):
    # Complète les trous
    img = morpho(tf.cast(image, tf.int16).numpy())
    # Au format BGR pour sauvegarde avec imwrite
    img = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_RGB2BGR)
    cv2.imwrite(dest_img+tf.compat.as_str_any(filename.numpy()),img)

### Vérification du détourage visuel

> Erreur sur certaines cellules, totalement noir => suppression des images de la base

In [14]:
# Récupère le nom de toutes les images sauvegardées => suppression dans la bd
import os
files=[]
for file in os.scandir(dest_img):
    files.append(file.name)

In [15]:
len(files)

31602

In [16]:
data[data['filename'].isin(files)].reset_index().to_csv('../files_detour_256.csv')